In [1]:
# !pip install boto3
# !pip install configparser
# !pip install pdftotext

#AWS
import boto3
import json
import os

#PDFTEXT
import urllib.request
import pdftotext

import re

## 1. Import pdf text from /Download folder

### In the folder Download you have a lot of pdf from ArXiv.com. 
### You can also import your own pdf in the folder "Teacher" and use the code below to extract the entities names

In [2]:
path='./Download'
first_article=os.listdir(path)[0]
last_article=os.listdir(path)[-1]

In [3]:
print(first_article)
print(last_article)

Assouad type dimensions of parabolic Julia sets.pdf
Temporal Difference Learning for Model Predictive Control.pdf


In [4]:
# Load your PDF
def load_pdf(article):
    try:
        with open(f'Download/{article}', "rb") as f:
            pdf = pdftotext.PDF(f)
            mypdftext="\n\n".join(pdf)
    except:
        with open(f'Teacher/{article}', "rb") as f:
            pdf = pdftotext.PDF(f)
            mypdftext="\n\n".join(pdf)
    return mypdftext

In [5]:
pdf=load_pdf(first_article)

In [6]:
#First filter to have only the References from a pdf
def after_references(mypdftext=load_pdf(first_article)): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    else:
        after_keyword = mypdftext[5000:]
    return after_keyword

In [7]:
#Display references in the pdf
references=after_references(pdf)
references

'\r\n[1] Aaronson, J., Denker, M., and Urbański, M. (1993). Ergodic theory for Markov fibred systems\r\nand parabolic rational maps. Trans. Amer. Math. Soc., 337(2):495–548.\r\n[2] Anderson, T. C., Hughes, K., Roos, J., and Seeger, A. (2021). Lp → Lq bounds for spherical\r\nmaximal operators. Math. Z., 297(3-4):1057–1074.\r\n[3] Beardon, A. F. (1991). Iteration of rational maps. Springer Verlag, New York.\r\n[4] Bishop, C. J. and Peres, Y. (2017). Fractals in Probability and Analysis, volume 162 of Cambridge\r\nStudies in Advanced Mathematics. Cambridge.\r\n[5] Bowen, R. (1979). Hausdorff dimension of quasi-circles. Pub. Math. IHES, 50:11–25.\r\n[6] Cheraghi, D., DeZotti, A., and Yang, F. (2020). Dimension paradox of irrationally indifferent\r\nattractors. preprint, available at: https://arxiv.org/abs/2003.12340.\r\n[7] Denker, M. and Urbański, M. (1991a). Absolutely continuous invariant measures for expansive\r\nrational maps with rationally indifferent periodic points. Forum Math.,

In [8]:
#First cleaning of this text

replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would'),
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns): 
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s) 
        return s

replacer=RegexpReplacer()

references=replacer.replace(references)
print(references)


[1] Aaronson, J., Denker, M., and Urbański, M. (1993). Ergodic theory for Markov fibred systems
and parabolic rational maps. Trans. Amer. Math. Soc., 337(2):495–548.
[2] Anderson, T. C., Hughes, K., Roos, J., and Seeger, A. (2021). Lp → Lq bounds for spherical
maximal operators. Math. Z., 297(3-4):1057–1074.
[3] Beardon, A. F. (1991). Iteration of rational maps. Springer Verlag, New York.
[4] Bishop, C. J. and Peres, Y. (2017). Fractals in Probability and Analysis, volume 162 of Cambridge
Studies in Advanced Mathematics. Cambridge.
[5] Bowen, R. (1979). Hausdorff dimension of quasi-circles. Pub. Math. IHES, 50:11–25.
[6] Cheraghi, D., DeZotti, A., and Yang, F. (2020). Dimension paradox of irrationally indifferent
attractors. preprint, available at: https://arxiv.org/abs/2003.12340.
[7] Denker, M. and Urbański, M. (1991a). Absolutely continuous invariant measures for expansive
rational maps with rationally indifferent periodic points. Forum Math., 3(6):561–580.
[8] Denker, M. and Urb

## 2. AWS Comprehend service

### If Cli doesn't works, insert your variable here and execute the cell below , otherwise skip this cell :

In [9]:
ACCESS_ID='...'
ACCESS_KEY='...'
session_token='...'

In [10]:
#With CLI
def compehend(references):
    try:
        comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
        input_text = references
        result_json=comprehend.detect_entities(Text=input_text, LanguageCode='en')
        print(result_json)
    except :
        comprehend = boto3.client(service_name='comprehend', region_name='us-east-1',aws_access_key_id=ACCESS_ID,
                                            aws_secret_access_key= ACCESS_KEY,aws_session_token=session_token)
        input_text = references
        result_json=comprehend.detect_entities(Text=input_text, LanguageCode='en')
        
    return result_json

In [11]:
result_json= compehend(references)
result_json

{'Entities': [{'Score': 0.9867931604385376, 'Type': 'PERSON', 'Text': 'Aaronson, J.', 'BeginOffset': 6, 'EndOffset': 18}, {'Score': 0.9666887521743774, 'Type': 'PERSON', 'Text': 'Denker, M.', 'BeginOffset': 20, 'EndOffset': 30}, {'Score': 0.9715626239776611, 'Type': 'PERSON', 'Text': 'Urbański, M.', 'BeginOffset': 36, 'EndOffset': 49}, {'Score': 0.9946287274360657, 'Type': 'DATE', 'Text': '1993', 'BeginOffset': 51, 'EndOffset': 55}, {'Score': 0.6216310262680054, 'Type': 'TITLE', 'Text': 'Ergodic theory for Markov', 'BeginOffset': 58, 'EndOffset': 83}, {'Score': 0.5866131782531738, 'Type': 'TITLE', 'Text': 'systems\r\nand', 'BeginOffset': 91, 'EndOffset': 103}, {'Score': 0.6200372576713562, 'Type': 'OTHER', 'Text': '337', 'BeginOffset': 154, 'EndOffset': 157}, {'Score': 0.7530454397201538, 'Type': 'OTHER', 'Text': '495–548', 'BeginOffset': 161, 'EndOffset': 168}, {'Score': 0.9599947333335876, 'Type': 'PERSON', 'Text': 'Anderson, T. C.', 'BeginOffset': 175, 'EndOffset': 190}, {'Score': 

{'Entities': [{'Score': 0.9867931604385376,
   'Type': 'PERSON',
   'Text': 'Aaronson, J.',
   'BeginOffset': 6,
   'EndOffset': 18},
  {'Score': 0.9666887521743774,
   'Type': 'PERSON',
   'Text': 'Denker, M.',
   'BeginOffset': 20,
   'EndOffset': 30},
  {'Score': 0.9715626239776611,
   'Type': 'PERSON',
   'Text': 'Urbański, M.',
   'BeginOffset': 36,
   'EndOffset': 49},
  {'Score': 0.9946287274360657,
   'Type': 'DATE',
   'Text': '1993',
   'BeginOffset': 51,
   'EndOffset': 55},
  {'Score': 0.6216310262680054,
   'Type': 'TITLE',
   'Text': 'Ergodic theory for Markov',
   'BeginOffset': 58,
   'EndOffset': 83},
  {'Score': 0.5866131782531738,
   'Type': 'TITLE',
   'Text': 'systems\r\nand',
   'BeginOffset': 91,
   'EndOffset': 103},
  {'Score': 0.6200372576713562,
   'Type': 'OTHER',
   'Text': '337',
   'BeginOffset': 154,
   'EndOffset': 157},
  {'Score': 0.7530454397201538,
   'Type': 'OTHER',
   'Text': '495–548',
   'BeginOffset': 161,
   'EndOffset': 168},
  {'Score': 0.

###  Export of entities in json

In [12]:
#Creation of json in order to keep only relevant information
def json_creation(result_json):
    output_json=[]
    i=0
    for json in result_json['Entities']:
        data={}
        if json['Type']== 'PERSON' and json['Score']>0.80:
            data["index"] = i
            data["Person"] = json['Text']
            data["Fiability"] = json['Score']
            output_json.append(data)
            i += 1
    return output_json

In [13]:
import json
import os

#Convert the array to json file
def create_json_file(jsonobject, filepath):
    with open(filepath, 'w') as outfile:
        json.dump(jsonobject, outfile)

In [14]:
#Import the json in my desktop
filename = "./Download/EntitiesPDF.json"
create_json_file(json_creation(result_json), filename)

#Display the json result
f = open("./Download/EntitiesPDF.json", "r")
print(f.read())

[{"index": 0, "Person": "Aaronson, J.", "Fiability": 0.9867931604385376}, {"index": 1, "Person": "Denker, M.", "Fiability": 0.9666887521743774}, {"index": 2, "Person": "Urban\u0301ski, M.", "Fiability": 0.9715626239776611}, {"index": 3, "Person": "Anderson, T. C.", "Fiability": 0.9599947333335876}, {"index": 4, "Person": "Hughes, K.", "Fiability": 0.9457246661186218}, {"index": 5, "Person": "Roos, J.", "Fiability": 0.9628537893295288}, {"index": 6, "Person": "Seeger, A.", "Fiability": 0.9617085456848145}, {"index": 7, "Person": "Beardon, A. F.", "Fiability": 0.9783458113670349}, {"index": 8, "Person": "Bishop, C. J.", "Fiability": 0.9074485301971436}, {"index": 9, "Person": "Peres, Y.", "Fiability": 0.9677588939666748}, {"index": 10, "Person": "Bowen, R.", "Fiability": 0.983867347240448}, {"index": 11, "Person": "Hausdorff", "Fiability": 0.8614009022712708}, {"index": 12, "Person": "Cheraghi, D.", "Fiability": 0.9779359698295593}, {"index": 13, "Person": "DeZotti, A.", "Fiability": 0.9

## 3. Test the code with your pdf

In [15]:
#HERE YOUR OWN PDF:
path='./Teacher/'
article_teacher=os.listdir(path)[-1]
print(article_teacher)

projetSIO2021-2022.pdf


In [20]:
my_article=load_pdf(article_teacher)
my_article=my_article[:5000]
print(my_article)

PROJET MODULE ARCHITECTURE SOA




                                 1


Documenter une solution et une API

• Ce projet est à réaliser par dessus un de vos projet fil rouge existant qui intègre une API
    • A priori celui de François Laissus en Réseaux,
    • Toute autre projet / API est accepté pourvu que l'on puisse le documenter et éventuellement le
      tester.

• Au minimum :
    • Documenter la solution en utilisant le formalisme C4 abordé en cours.
    • Documenter avec le standard OpenAPI la ou les API de votre projet. La documentation produite
      doit donner tout le fonctionnement, des exemples et un modèle clair, tant sur les cas nominaux
      que les erreurs.
• Bonus potentiel :
    • Proposer une version "managée" de l'API au travers d'un API Manager pour offrir des adaptations
      SANS CHANGER le code de votre solution


Objectifs

• Documenter l’architecture
   • Formaliser avec au moins les niveaux Contexte ET Conteneur C4.
• Open API
   • Documenter une API po

In [17]:
#AWS comprehend
result_json=compehend(my_article)

{'Entities': [{'Score': 0.3493761420249939, 'Type': 'TITLE', 'Text': '1', 'BeginOffset': 73, 'EndOffset': 74}, {'Score': 0.680497407913208, 'Type': 'QUANTITY', 'Text': 'un de vos projet', 'BeginOffset': 155, 'EndOffset': 171}, {'Score': 0.9977027177810669, 'Type': 'PERSON', 'Text': 'François Laissus', 'BeginOffset': 236, 'EndOffset': 252}, {'Score': 0.6706495881080627, 'Type': 'QUANTITY', 'Text': 'Toute autre projet', 'BeginOffset': 272, 'EndOffset': 290}, {'Score': 0.45497217774391174, 'Type': 'OTHER', 'Text': 'C4', 'BeginOffset': 457, 'EndOffset': 459}, {'Score': 0.8793732523918152, 'Type': 'TITLE', 'Text': 'OpenAPI', 'BeginOffset': 512, 'EndOffset': 519}, {'Score': 0.7060993909835815, 'Type': 'QUANTITY', 'Text': 'tout le fonctionnement', 'BeginOffset': 596, 'EndOffset': 618}, {'Score': 0.650689423084259, 'Type': 'TITLE', 'Text': 'API', 'BeginOffset': 787, 'EndOffset': 790}, {'Score': 0.37040069699287415, 'Type': 'OTHER', 'Text': 'C4', 'BeginOffset': 983, 'EndOffset': 985}, {'Score':

In [18]:
#Creation of json in order to keep only relevant information
import json
import os

#Import the json in my desktop
filename = "./Teacher/AWS_json_teacher.json"
create_json_file(json_creation(result_json), filename)

#Display the json result
f = open("./Teacher/AWS_json_teacher.json", "r")
print(f.read())

[{"index": 0, "Person": "Fran\u00e7ois Laissus", "Fiability": 0.9977027177810669}]


## You can go in the folder Download or Teacher to see the jsonfile